In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# 장르 속성을 이용한 영화 콘텐츠 기반 필터링

## 데이터 로딩 및 가공

In [2]:
movies = pd.read_csv('./tmdb_5000_movies.csv')

In [3]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [4]:
movies['genres']

0       [{"id": 28, "name": "Action"}, {"id": 12, "nam...
1       [{"id": 12, "name": "Adventure"}, {"id": 14, "...
2       [{"id": 28, "name": "Action"}, {"id": 12, "nam...
3       [{"id": 28, "name": "Action"}, {"id": 80, "nam...
4       [{"id": 28, "name": "Action"}, {"id": 12, "nam...
                              ...                        
4798    [{"id": 28, "name": "Action"}, {"id": 80, "nam...
4799    [{"id": 35, "name": "Comedy"}, {"id": 10749, "...
4800    [{"id": 35, "name": "Comedy"}, {"id": 18, "nam...
4801                                                   []
4802                  [{"id": 99, "name": "Documentary"}]
Name: genres, Length: 4803, dtype: object

## 분석에 사용할 주요 컬럼 추출
- id, title, generes, vote_average(평점), vote_count(펑점 투표 수), popularity (영화 인기도), keyword, overview(영화개요)

In [5]:
movies.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

In [6]:
movies_df=movies[['id','title','genres','vote_average','vote_count','popularity','keywords','overview']]

In [7]:
movies_df[['genres','keywords']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   genres    4803 non-null   object
 1   keywords  4803 non-null   object
dtypes: object(2)
memory usage: 75.2+ KB


In [8]:
# import re
# p = re.compile('(?<=\"name": ")[A-Za-z]+')
# movies['genres'] = movies['genres'].apply(p.findall)
# movies['genres'] 

- eval()
    - 문자형태로 되어있는 표현식을 실행하는 함수로, 함수나 객체도 실행가능하나 literal_eval은 파이썬에서 제공하는 기본 데이터 타입 정도만 변환해주는 용도로 사용 가능하다.

In [9]:
eval('1+(2+3)*2')

11

- literal_eval() 함수를 통해 'genres' 컬럼과 'keywords'컬럼의 값을 list 객체로 변환

In [10]:
from ast import literal_eval
movies_df['genres'] = movies_df['genres'].apply(literal_eval)
#장르 컬럼에 있는 것을 literal_eval에 넣음
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)

In [11]:
movies_df['genres'].values[0]

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'}]

In [12]:
movies_df['keywords'].values[0]

[{'id': 1463, 'name': 'culture clash'},
 {'id': 2964, 'name': 'future'},
 {'id': 3386, 'name': 'space war'},
 {'id': 3388, 'name': 'space colony'},
 {'id': 3679, 'name': 'society'},
 {'id': 3801, 'name': 'space travel'},
 {'id': 9685, 'name': 'futuristic'},
 {'id': 9840, 'name': 'romance'},
 {'id': 9882, 'name': 'space'},
 {'id': 9951, 'name': 'alien'},
 {'id': 10148, 'name': 'tribe'},
 {'id': 10158, 'name': 'alien planet'},
 {'id': 10987, 'name': 'cgi'},
 {'id': 11399, 'name': 'marine'},
 {'id': 13065, 'name': 'soldier'},
 {'id': 14643, 'name': 'battle'},
 {'id': 14720, 'name': 'love affair'},
 {'id': 165431, 'name': 'anti war'},
 {'id': 193554, 'name': 'power relations'},
 {'id': 206690, 'name': 'mind and soul'},
 {'id': 209714, 'name': '3d'}]

- 장르와 키워드 컬럼의 값(name 키의 값)만 원소로 추출하여 리스트로 생성

In [13]:
movies_df['genres'] = movies_df['genres'].apply(lambda x : [v['name'] for v in x])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x : [v['name'] for v in x])

## 장르 콘텐츠 유사도 측정
- 문자열로 반환된 'genres'컬럼은 카운트 기반으로 피처 백터화 변환 (사이킷런의 CountVectorizer 클래스를 이용)
- 'genres'문자열을 피쳐 백터화 행렬로 변환한 데이터 셋을 코사인 유사도를 통해 비교한다.
- genres 유사도가 높은 연화중에 평점이 높은 순으로 영화를 추천

In [14]:
movies_df['genres']

0       [Action, Adventure, Fantasy, Science Fiction]
1                        [Adventure, Fantasy, Action]
2                          [Action, Adventure, Crime]
3                    [Action, Crime, Drama, Thriller]
4                [Action, Adventure, Science Fiction]
                            ...                      
4798                        [Action, Crime, Thriller]
4799                                [Comedy, Romance]
4800               [Comedy, Drama, Romance, TV Movie]
4801                                               []
4802                                    [Documentary]
Name: genres, Length: 4803, dtype: object

**[참고] CountVectorizer 의 사용법**
- 텍스트에서 단위별 출현 횟수를 카운팅하여 수치 백터화 한다.

In [15]:
from sklearn.feature_extraction.text import CountVectorizer
# ngram_range : 모델의 단어 순서를 보강하기 위한 범위 (범위 최솟값, 범위 최댓값)
#(1.1) 단어 1개씩 피처로 추출
#(1,2) 토큰화된 단어를 1개 씩, 그리고 순서대로 2개 묶어서 피처로 추출
vectorizer = CountVectorizer(ngram_range=(1,1))
#4개의 어휘를 학습한 CountVectorizer를 생성

vectorizer.fit(['첫번째 문서 테스트','두번째 문서 테스트'])
print(vectorizer.vocabulary_)

{'첫번째': 2, '문서': 1, '테스트': 3, '두번째': 0}


In [16]:
#문자열 목록을 가져와 미리 학습 해놓은 사전을 기반으로 단어의 빈도수를 세어준다.
count = vectorizer.transform(['직접 첫번째 테스트 두번째 테스트'])
print(count.toarray())
# 학습된 인덱스에 따라 단어 수를 세어줌

[[1 0 1 2]]


In [17]:
movies_df['genres_text'] = movies_df['genres'].apply(lambda x : (' ').join(x) )
movies_df['genres_text']

count_vect = CountVectorizer(min_df=0, ngram_range=(1,2)) 
#min_df = 낮은 빈도수 제외하는 파라미터
#min_df = 2일 경우 전체 문서에서 2번 이하로 나타나는 단어는 추출에서 제외
genre_mat = count_vect.fit_transform(movies_df['genres_text'])
print(genre_mat.shape)
#4803: 영화의 개수 / 276: CountVectorizer에 의해 생성된 장르 단어 목록의 개수

(4803, 276)


In [37]:
movies_df['genres_text'].values[0]

'Action Adventure Fantasy Science Fiction'

In [18]:
print(genre_mat.toarray()[:1])

[[1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


In [19]:
from sklearn.metrics.pairwise import cosine_similarity
# -1<코사인 유사도<1 --> 대각 행렬을 기준으로 좌우 대칭 ( 대각행렬은 1, 자기 자신에 대한 유사도)
# 기준행, 비교
genre_sim = cosine_similarity(genre_mat)
print(genre_sim.shape)
print(genre_sim)

(4803, 4803)
[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]
 [0.59628479 1.         0.4        ... 0.         0.         0.        ]
 [0.4472136  0.4        1.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


In [20]:
print(genre_sim[:3])

[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]
 [0.59628479 1.         0.4        ... 0.         0.         0.        ]
 [0.4472136  0.4        1.         ... 0.         0.         0.        ]]


In [21]:
# 첫번째 영화와 유사도가 높은 값의 인덱스

In [22]:
genre_sim_sorted_idx = genre_sim.argsort()[:,::-1]
# argsort는 행단위 정렬
#첫번째 영화와 장르 유사도가 높은 영화의 movie_df 상에서 index 값
print(genre_sim_sorted_idx.shape)
print(genre_sim_sorted_idx[:1])
# 자기 자신을 나타내는 0번 래코드를 제외하면 3494번째 영화가 장르 기준으로 가장 유사도가 높은 영화이다.

(4803, 4803)
[[   0 3494  813 ... 3038 3037 2401]]


In [23]:
genre_sim_sorted_idx[0,1:11]

array([3494,  813,  870,   46,   14, 1296, 1652,  419,  420, 1191],
      dtype=int64)

In [24]:
movies_df.head()

,id,title,genres,vote_average,vote_count,popularity,keywords,overview,genres_text
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]",7.2,11800,150.437577,"[culture clash, future, space war, space colon...","In the 22nd century, a paraplegic Marine is di...",Action Adventure Fantasy Science Fiction
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]",6.9,4500,139.082615,"[ocean, drug abuse, exotic island, east india ...","Captain Barbossa, long believed to be dead, ha...",Adventure Fantasy Action
2,206647,Spectre,"[Action, Adventure, Crime]",6.3,4466,107.376788,"[spy, based on novel, secret agent, sequel, mi...",A cryptic message from Bond’s past sends him o...,Action Adventure Crime
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller]",7.6,9106,112.312950,"[dc comics, crime fighter, terrorist, secret i...",Following the death of District Attorney Harve...,Action Crime Drama Thriller
4,49529,John Carter,"[Action, Adventure, Science Fiction]",6.1,2124,43.926995,"[based on novel, mars, medallion, space travel...","John Carter is a war-weary, former military ca...",Action Adventure Science Fiction


In [25]:
print(movies_df[movies_df['title']=='Avatar'].index.values)
print(type(movies_df[movies_df['title']=='Avatar'].index.values))

[0]
<class 'numpy.ndarray'>


## 장르 콘텐츠 필터링을 이용한 영화추천

- 좀 더 많은 후보군을 선정한 뒤에 영화의 평점에 따라 필터링 하는 방식으로 변경이 필요

In [26]:
movies_df[['title', 'vote_average','vote_count']].sort_values('vote_average', ascending= False)[:10]

,title,vote_average,vote_count
3519,Stiff Upper Lips,10.0,1
4247,Me You and Five Bucks,10.0,2
4045,"Dancer, Texas Pop. 81",10.0,1
4662,Little Big Top,10.0,1
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
2970,There Goes My Baby,8.5,2
1881,The Shawshank Redemption,8.5,8205
2796,The Prisoner of Zenda,8.4,11
3337,The Godfather,8.4,5893


- 왜곡된 평점 데이터를 피하기 위해 평점에 평가 횟수를 반영한 가중 평점 방식 적용 (IMDB에서 평점을 매기는 방식)
- 가중평점 (Weighted Rating) = (v/(v+m))*R + (m/(v+m)) *C
    - v: 개별 영화에 평점을 투표한 횟수
    - m: 평점을 부여하기 위한 최소 투표 횟수(전체 투표 횟수에서 상위 60%에 해당하는 횟수를 기준으로 정함<임의값>)
    - R: 개별 영화에 대한 평균 평점
    - C: 전체 영화에 대한 평균 평점
    

In [27]:
C = movies_df['vote_average'].mean()
m = movies_df['vote_count'].quantile(0.6)
print(f'C:{C:.3f}, m:{m:.3f}')

C:6.092, m:370.200


In [28]:
def weighted_vote_average(data):
    v = data['vote_count']
    R = data['vote_average']
    
    return (v/(v+m))*R + (m/(v+m)) *C



In [29]:
movies_df.apply(weighted_vote_average, axis=1)

0       7.166301
1       6.838594
2       6.284091
3       7.541095
4       6.098838
          ...   
4798    6.290894
4799    6.089611
4800    6.106650
4801    6.084894
4802    6.100782
Length: 4803, dtype: float64

In [30]:
movies_df['weighted_vote'] = movies_df.apply(weighted_vote_average, axis=1)

In [31]:
movies_df.head()

,id,title,genres,vote_average,vote_count,popularity,keywords,overview,genres_text,weighted_vote
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]",7.2,11800,150.437577,"[culture clash, future, space war, space colon...","In the 22nd century, a paraplegic Marine is di...",Action Adventure Fantasy Science Fiction,7.166301
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]",6.9,4500,139.082615,"[ocean, drug abuse, exotic island, east india ...","Captain Barbossa, long believed to be dead, ha...",Adventure Fantasy Action,6.838594
2,206647,Spectre,"[Action, Adventure, Crime]",6.3,4466,107.376788,"[spy, based on novel, secret agent, sequel, mi...",A cryptic message from Bond’s past sends him o...,Action Adventure Crime,6.284091
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller]",7.6,9106,112.312950,"[dc comics, crime fighter, terrorist, secret i...",Following the death of District Attorney Harve...,Action Crime Drama Thriller,7.541095
4,49529,John Carter,"[Action, Adventure, Science Fiction]",6.1,2124,43.926995,"[based on novel, mars, medallion, space travel...","John Carter is a war-weary, former military ca...",Action Adventure Science Fiction,6.098838


In [32]:
movies_df[['title','vote_average','weighted_vote','vote_count']].sort_values('weighted_vote',ascending=False)[:10]

,title,vote_average,weighted_vote,vote_count
1881,The Shawshank Redemption,8.5,8.396052,8205
3337,The Godfather,8.4,8.263591,5893
662,Fight Club,8.3,8.216455,9413
3232,Pulp Fiction,8.3,8.207102,8428
65,The Dark Knight,8.2,8.136930,12002
1818,Schindler's List,8.3,8.126069,4329
3865,Whiplash,8.3,8.123248,4254
809,Forrest Gump,8.2,8.105954,7927
2294,Spirited Away,8.3,8.105867,3840
2731,The Godfather: Part II,8.3,8.079586,3338


In [33]:
# df: 영화정보를 가지고 있는 DataFrame
# sorted_ind : 장르 유사도가 높은 영화의 인덱스값
# title_name : 추천 기준이 되는 영화 제목(해당 연화와 장르 유사도가 높은 영화를 추천)
# top_n: 추천 영화 수 (유사도가 높은 상위 n개 영화)
def fin_sim_movie(df, sorted_ind, title_name, top_n=10):
    target_movie = df[df['title'] == title_name]
    title_index = target_movie.index.values
    # sorted_ind 인자로 입력된 genre_sim_sorted_idx 객체에서 유사도가 높은 top_n개의 영화 index를 추출
    
    # top_n의 2배 --> 후보군을 넓힘
    similar_indexes = sorted_ind[title_index, :(top_n*2)]
    similar_indexes = similar_indexes.reshape(-1)
    # 기준이 되는 영화 제외
    similar_indexes = similar_indexes[similar_indexes!=title_index]
    
    return df.iloc[similar_indexes].sort_values('weighted_vote',ascending = False)[:top_n]

In [34]:
genre_sim_sorted_idx

array([[   0, 3494,  813, ..., 3038, 3037, 2401],
       [ 262,    1,  129, ..., 3069, 3067, 2401],
       [   2, 1740, 1542, ..., 3000, 2999, 2401],
       ...,
       [4800, 3809, 1895, ..., 2229, 2230,    0],
       [4802, 1594, 1596, ..., 3204, 3205,    0],
       [4802, 4710, 4521, ..., 3140, 3141,    0]], dtype=int64)

In [35]:
genre_sim_sorted_idx[0, :(10*2)]

array([   0, 3494,  813,  870,   46,   14, 1296, 1652,  419,  420, 1191,
        238, 3208, 1932,   72,   10,   61,  232, 2592, 1802], dtype=int64)

In [36]:
sim_movies = fin_sim_movie(movies_df,genre_sim_sorted_idx,'The Godfather', 10)
sim_movies[['title', 'vote_average','weighted_vote']]

,title,vote_average,weighted_vote
2731,The Godfather: Part II,8.3,8.079586
1847,GoodFellas,8.2,7.976937
3866,City of God,8.1,7.759693
1663,Once Upon a Time in America,8.2,7.657811
883,Catch Me If You Can,7.7,7.557097
281,American Gangster,7.4,7.141396
4041,This Is England,7.4,6.739664
1149,American Hustle,6.8,6.717525
1243,Mean Streets,7.2,6.626569
2839,Rounders,6.9,6.530427
